<a href="https://colab.research.google.com/github/AgustinBustos/Tesis_Bella_Bonilla_Bustos/blob/main/kernelandtorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# principio

In [ ]:
import pandas as pd
import numpy as np
import torch
import plotly.express as px

In [ ]:
def f(x):
  return np.cos(x)
def splitToCV(xtrain,ytrain,lenght,valPerc):
  qval=int(lenght*valPerc)
  idxs=np.random.RandomState(seed=1523).permutation(lenght)
  xv=xtrain[idxs[:qval]]
  xt=xtrain[idxs[qval:]]
  yv=ytrain[idxs[:qval]]
  yt=ytrain[idxs[qval:]]
  return {'xt':xt,'xv':xv,'yt':yt,'yv':yv,'indexV':idxs[:qval],'indexT':idxs[qval:]}

In [ ]:
X=np.linspace(0,10,num=500)
y=f(X)+np.random.normal(scale=0.5,size=len(X))




px.scatter(x=X,y=y).show()

In [ ]:
xtorch=torch.from_numpy(X).float()
ytorch=torch.from_numpy(y).float()
xtorch=torch.reshape(xtorch,(-1,1))
ytorch=torch.reshape(ytorch,(-1,1))

data=splitToCV(X,y,len(X),0.5)
xt=data['xt']
yt=data['yt']
xv=data['xv']
yv=data['yv']

px.scatter(x=xv,y=yv).show()

xt=torch.from_numpy(xt).float()
yt=torch.from_numpy(yt).float()
xt=torch.reshape(xt,(-1,1))
yt=torch.reshape(yt,(-1,1))

xv=torch.from_numpy(xv).float()
yv=torch.from_numpy(yv).float()
xv=torch.reshape(xv,(-1,1))
yv=torch.reshape(yv,(-1,1))



# nn kernel

In [ ]:
l=torch.tensor(1.0)
def kernel(x):
  return torch.exp((-(x ** 2))/(2* l ** 2))
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  

px.line(x=xtorch.detach().numpy().reshape(-1),y=kernel(xtorch).detach().numpy().reshape(-1)).show()

In [ ]:
l=torch.tensor(3.0)
N=len(xt)
alpha=torch.tensor(1.0,requires_grad=True)
def model(x):
  lenght=len(x)
  iz=torch.ones(lenght,1) @ xt.t()
  der=x @ torch.ones(1,N)
  diff=iz-der
  
  return (kernel(diff) @ yt)*alpha
real=pd.DataFrame({'x':X,'y':y,'real':1})
fake=pd.DataFrame({'x':X,'y':model(xtorch).detach().numpy().reshape(-1),'real':0})
df=pd.concat([real,fake])
px.scatter(df,x='x',y='y',color='real').show()

## alpha auto BOLUDEZ

In [ ]:
l=torch.tensor(0.5)
N=len(xt)
alpha=torch.tensor(1.0,requires_grad=True)
def model(x):
  lenght=len(x)
  iz=torch.ones(lenght,1) @ xt.t()
  der=x @ torch.ones(1,N)
  diff=iz-der
  ponder=kernel(diff)
  divisores=torch.sum(ponder, dim=1)
  
  print(ponder)
  
  return (ponder @ yt)/torch.reshape(divisores,(-1,1))
model(xtorch[4:6])
# real=pd.DataFrame({'x':X,'y':y,'real':1})
# fake=pd.DataFrame({'x':X,'y':model(xtorch).detach().numpy().reshape(-1),'real':0})
# df=pd.concat([real,fake])
# px.scatter(df,x='x',y='y',color='real').show()

tensor([[1.2379e-01, 1.5655e-11, 0.0000e+00, 3.5140e-05, 4.9364e-34, 3.7181e-26,
         2.4400e-35, 0.0000e+00, 9.6120e-02, 0.0000e+00, 4.6214e-01, 9.4810e-06,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 4.1665e-27, 1.4537e-01, 3.3301e-01,
         3.7256e-02, 7.9340e-20, 6.7913e-37, 9.4989e-01, 5.2271e-22, 2.5233e-21,
         1.0596e-18, 0.0000e+00, 9.1234e-24, 0.0000e+00, 1.0321e-06, 0.0000e+00,
         1.1156e-03, 8.5465e-03, 0.0000e+00, 4.1257e-02, 6.6914e-35, 6.0532e-01,
         1.4926e-16, 2.1110e-14, 0.0000e+00, 2.8983e-08, 0.0000e+00, 9.3701e-01,
         1.2038e-04, 0.0000e+00, 1.8612e-17, 2.2869e-08, 0.0000e+00, 2.1903e-18,
         0.0000e+00, 2.9319e-29, 0.0000e+00, 5.3090e-17, 1.0048e-26, 1.5782e-22,
         0.0000e+00, 0.0000e+00, 1.1660e-29, 0.0000e+00, 3.1875e-25, 1.5548e-02,
         7.1221e-40, 6.7071e-02, 4.6878e-13, 0.0000e+00, 1.2922e-03, 0.0000e+00,
         5.2591e-42, 1.1521e-21, 1.1516e-12, 0.0000e+00, 0.0000e+00, 1.7414e-20,
         7.2522e-01, 1.9580e

tensor([[0.7006],
        [0.6967]])

In [ ]:
l=torch.tensor(0.5,requires_grad=True)
def kernel(x):
  return torch.exp((-(x)**2)/(2* l ** 2))  #no elevo x al cuadrado porque saco la norma sin raiz
#ojo que no uso alpha
alpha=torch.tensor(1.0,requires_grad=True)
def kreg(x):
  lenght=len(x)
  iz=torch.stack([torch.kron(torch.ones([len(xt),1]), row) for row in x])
  der=torch.stack([xt for i in range(lenght)])
  diff=iz-der
  norma=torch.norm(diff,dim=2)
  ponder=kernel(norma)
  divisores=torch.sum(ponder, dim=1)
  
  return (ponder @ yt)/torch.reshape(divisores,(-1,1))

real=pd.DataFrame({'x':X,'y':y,'real':1})
fake=pd.DataFrame({'x':X,'y':kreg(xtorch).detach().numpy().reshape(-1),'real':0})
df=pd.concat([real,fake])
px.scatter(df,x='x',y='y',color='real').show()

## alpha as parameter IMPORTANTE

In [ ]:
l=torch.tensor(3.0)
N=len(xt)
alpha=torch.tensor(1.0,requires_grad=True)
def model(x):
  lenght=len(x)
  iz=torch.ones(lenght,1) @ xt.t()
  der=x @ torch.ones(1,N)
  diff=iz-der
  
  return (kernel(diff) @ yt)*alpha

optimizer = torch.optim.Adam([alpha], lr=0.01)
for epoch in range(10000):
    yhat = model(xt)
    loss = mse(yhat, yt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

real=pd.DataFrame({'x':X,'y':y,'real':1})
fake=pd.DataFrame({'x':X,'y':model(xtorch).detach().numpy().reshape(-1),'real':0})
df=pd.concat([real,fake])
px.scatter(df,x='x',y='y',color='real').show()
